<a href="https://colab.research.google.com/github/Rattapon-Insa/time-series/blob/main/Multivariate_Time_series_on_NIFTY_50_Stock_Market_Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! mkdir ~/.kaggle

In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d rohanrao/nifty50-stock-market-data

In [ ]:
! unzip nifty50-stock-market-data.zip

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
df = pd.read_csv('HCLTECH.csv')

In [ ]:
df['Date'] =  pd.to_datetime(df['Date'])
df.dropna()
df.set_index(df['Date'], inplace = True)

In [ ]:
df2 = df[['Open','High','Low','Close']]['2013-01-01':'2013-12-02']

In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

df2_traing_scaled = sc.fit_transform(df2)


In [ ]:
sc2 = StandardScaler()
df2_traing_scaled_y = sc2.fit_transform(df2[['Open']])
df2_traing_scaled_y

In [ ]:
df2_traing_scaled.shape

In [ ]:
hops = 10
total_record = df2_traing_scaled.shape[0]
X = []
Y = []
for i in range(hops, total_record):
  X.append(df2_traing_scaled[i-hops:i])
  Y.append(df2_traing_scaled_y[i][0])
X, Y = np.array(X), np.array(Y)

In [ ]:
X.shape

In [ ]:
Y.shape

In [ ]:
X_shaped = np.reshape(X,(X.shape[0], X.shape[1], X.shape[2]))
X_shaped.shape

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout

model = Sequential()
model.add(LSTM(units = 50, return_sequences= True,
               input_shape = (hops,4))) # 4 come from 4 features
model.add(Dropout(0.2))
model.add(LSTM(units = 50))
model.add(Dropout(0.2))
model.add(Dense(1))
model.compile(optimizer = 'adam', loss = 'mean_squared_error', metrics = 'mae')

In [ ]:
model.fit(X_shaped, Y, epochs= 100, batch_size = 32)